In [1]:
import sys
import os
import timeit

current_dir = os.getcwd()

parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm import *
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import gridspec
import random

from Networks.DNN import PINN
from Networks.IA_DNN import Attention_PINN
from Networks.Fourier_DNN import Fourier_PINN
from Networks.AF_PINN import AF_PINN
from Networks.Efficient_KAN import KAN
from Networks.Cheby_KAN import ChebyKAN
from DataGenerator import DataGenerator

import pandas as pd

from torch.utils.data import TensorDataset, DataLoader

import time
from datetime import datetime

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
def seed_torch(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
seed_torch(42)

Using device: cuda:0


In [2]:
L = 600.0
h = 100.0
anneal_time= 60
D_A = 360.0  #6nm^2/s 360nm^2/min
k11,k12,k21 = 3.6e-3,3.6e-3,0.0
N_SA,N_SB,N_SC,N_SAB,N_SABB,N_SAAB = 90.0,48.0,90.0,90.0,90.0,90.0

geom = [0,L]
time = [0,anneal_time]

##How I genertate data
generator = DataGenerator(geom,time)

from pde_system import compute_pde, compute_bc, compute_ic,relative_error
from graphics import plot_loss_curve,plot_relative_concen,plot_3d_concen
# from Utilitiy.residual_adaptive_resample import select_high_loss_points


In [3]:
from torch import Tensor


hiddens = [6, 5, 4, 3]  ##[3,4] already finished
neurons = [128,96,64]

activation = torch.nn.functional.silu

start_lr = 1e-3
end_lr = 1e-8

weight = [1, 1, 1, 1, 1, 1]
for hidden in hiddens:
    for neuron in neurons:
        for mode in ['PINN']:  ##['PINN', 'IA_PINN', 'Fourier_PINN', 'AF_PINN']
            layer = [2] + [neuron]*hidden + [6]
            
            star_time = timeit.default_timer()
            num_epochs = 5000
            sub_spochs = 10

            loss_log_total,loss_log_pde, loss_log_bc, loss_log_ic = [],[],[],[]
            torch.cuda.empty_cache()
            print(f'{mode}_[{neuron}]x{hidden}:')
            
            if mode == 'PINN':
                model = PINN(layer,activation,non_negative=True).to(device)
                optimizer = optim.Adam(model.parameters(), lr=start_lr)  
                scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * sub_spochs, eta_min=end_lr)
            elif mode == 'IA_PINN':
                model = Attention_PINN(layer,activation,non_negative=True).to(device)
                optimizer = optim.Adam(model.parameters(), lr=start_lr)  
                scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * sub_spochs, eta_min=end_lr)
            elif mode == 'Fourier_PINN':
                model = Fourier_PINN(layer,non_negative=True, activation= activation, use_rff=True, rff_num_features=64, rff_sigma=5.0).to(device)
                optimizer = optim.Adam(model.parameters(), lr=start_lr)  
                scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * sub_spochs, eta_min=end_lr)
            elif mode == 'AF_PINN':
                model = AF_PINN(layer, non_negative=True,activation = activation, use_rff=True, rff_num_features=64, rff_sigma=5.0).to(device)
                optimizer = optim.Adam(model.parameters(), lr=start_lr)  
                scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * sub_spochs, eta_min=end_lr)
            else:
                print("Man! What can I say! Wrong Mode!!!")
                break            
            points_types = ['domain', 'left_bc', 'right_bc', 'initial']

            coords_pde, coords_bc1, coords_bc2, coords_ic = [
                generator.grid_generator(2500 if pt in ['left_bc', 'right_bc'] else 10201, points_type=pt, nondim= True) 
                for pt in points_types
            ]

            for idx in range(sub_spochs):
                print(f'Epoch: {idx+1}/{sub_spochs}----------------')
                model.train()
                
                pbar = tqdm(range(num_epochs))
                for epoch in pbar:
                    current_lr = optimizer.param_groups[0]['lr'] 
                    def closure() -> Tensor:
                        global loss_pde, D_star, loss_bc1, loss_bc2, loss_ic1, loss_ic2
                        optimizer.zero_grad()

                        # 计算损失时使用batch数据
                        loss_eq1, loss_eq2, loss_eq3, loss_eq4, loss_eq5, loss_eq6, D_star = compute_pde(model, coords_pde)
                        loss_pde = loss_eq1 + loss_eq2 + loss_eq3 + loss_eq4 + loss_eq5 + loss_eq6
                        loss_bc1 = compute_bc(model, coords_bc1)
                        loss_bc2 = compute_bc(model, coords_bc2)
                        loss_ic1, loss_ic2 = compute_ic(model, coords_ic)

                        #Give a weight to initial loss term
                        loss = weight[0]*loss_pde +  weight[1]*loss_bc1 +  weight[2]*loss_bc2 +  weight[3]*loss_ic1 +  weight[4]*loss_ic2 

                        loss.backward()
                        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0, norm_type= 2)
                        return loss

                    optimizer.step(closure)
                    
                    # 如果使用scheduler（只在Adam阶段）
                    if scheduler is not None:
                        scheduler.step()
                    
                    loss_total = weight[0]*loss_pde +  weight[1]*loss_bc1 +  weight[2]*loss_bc2 +  weight[3]*loss_ic1 +  weight[4]*loss_ic2 

                    loss_log_total.append(loss_total.item())
                    loss_log_pde.append(loss_pde.item())
                    loss_log_bc.append(loss_bc1.item() + loss_bc2.item())
                    loss_log_ic.append(loss_ic1.item() + loss_ic2.item())
                    
                    if epoch % 10 == 0:
                        pbar.set_postfix({
                            'Loss': '{0:.3e}'.format(loss_total),
                            'PDE': '{0:.3e}'.format(weight[0]*loss_pde.item()),
                            'BC': '{0:.3e},{1:.3e}'.format(weight[1]*loss_bc1.item(),weight[2]*loss_bc2.item()),
                            'IC': '{0:.3e},{1:.3e}'.format(weight[3]*loss_ic1.item(),weight[4]*loss_ic2.item()),
                            'D*': '{0:.2e},{1:.2e}'.format(torch.max(D_star),torch.min(D_star)),
                            'lr': '{0:.2e}'.format(current_lr),
                        })        
                    # # 将四个张量放在一个列表中并随机打乱
                    shuffled_tensors = [tensor[torch.randperm(tensor.size(0))] for tensor in [coords_pde, coords_bc1, coords_bc2, coords_ic]]

                    # 解包到四个变量中
                    coords_pde, coords_bc1, coords_bc2, coords_ic = shuffled_tensors
                
                    if torch.isnan(loss_total):
                        print('Loss is Nan, please check your model parameters')
                        break
                
                true_path = './FDM/range[0,1]/'
                file_names = ['u1_output.csv','u2_output.csv','u3_output.csv','u4_output.csv','u5_output.csv','u6_output.csv']
                true_data = [pd.read_csv(true_path + file_name,header=None) for file_name in file_names]

                model.eval()
                val_data = generator.grid_generator(N=10201, points_type= 'domain', nondim= True)
                result = model(val_data).data.cpu().numpy()
                relative_error(result,true_data)

                ##模型储存
                save_path = f'./Results/{mode}_[{neuron}]x{hidden}/model_save/'
                os.makedirs(save_path, exist_ok=True)  # 确保保存路径存在
                # 指定模型文件名
                model_file = os.path.join(save_path, f'{mode}_[{neuron}]x{hidden}_{(idx+1)*num_epochs}.pth')
                # 保存模型
                torch.save(model.state_dict(), model_file)
                print(f"模型已保存至 {model_file}")
            
            # 绘制结果
            plot_loss_curve(f'{mode}_[{neuron}]x{hidden}',loss_log_total,loss_log_pde, loss_log_bc, loss_log_ic)
            
            plot_relative_concen(f'{mode}_[{neuron}]x{hidden}',result,data_save=True)
            
            plot_3d_concen(f'{mode}_[{neuron}]x{hidden}', result, true_data)

            Training_time = timeit.default_timer() - star_time
            print(f'Training time: {Training_time:.2f}s-------------------------')


PINN_[128]x6:
Epoch: 1/10----------------


100%|██████████| 5000/5000 [13:23<00:00,  6.22it/s, Loss=1.296e-02, PDE=3.972e-03, BC=1.735e-04,2.007e-04, IC=8.041e-03,5.774e-04, D*=6.00e-02,3.94e-03, lr=9.76e-04]


R^2 Error: {'Ni': 0.068, 'SiC': 0.022, 'C': 0.086, 'NiSi': 0.198, 'NiSi2': 0.192}
Inf Error: {'Ni': 0.066, 'SiC': 0.019, 'C': 0.166, 'NiSi': 0.28, 'NiSi2': 0.269}
Mean Error: (0.11299526790560555, 0.16011207649065823)
模型已保存至 ./Results/PINN_[128]x6/model_save/PINN_[128]x6_5000.pth
Epoch: 2/10----------------


100%|██████████| 5000/5000 [13:38<00:00,  6.11it/s, Loss=9.551e-03, PDE=2.526e-03, BC=3.816e-04,1.296e-05, IC=6.117e-03,5.133e-04, D*=6.00e-02,2.99e-03, lr=9.05e-04]


R^2 Error: {'Ni': 0.06, 'SiC': 0.015, 'C': 0.062, 'NiSi': 0.172, 'NiSi2': 0.201}
Inf Error: {'Ni': 0.055, 'SiC': 0.017, 'C': 0.142, 'NiSi': 0.288, 'NiSi2': 0.392}
Mean Error: (0.10198799662802471, 0.17902898882330737)
模型已保存至 ./Results/PINN_[128]x6/model_save/PINN_[128]x6_10000.pth
Epoch: 3/10----------------


...

R^2 Error: {'Ni': 0.075, 'SiC': 0.011, 'C': 0.058, 'NiSi': 0.102, 'NiSi2': 0.052}
Inf Error: {'Ni': 0.072, 'SiC': 0.018, 'C': 0.116, 'NiSi': 0.111, 'NiSi2': 0.062}
Mean Error: (0.05980673738456509, 0.07581488104220341)
模型已保存至 ./Results/PINN_[64]x3/model_save/PINN_[64]x3_40000.pth
Epoch: 9/10----------------


100%|██████████| 5000/5000 [07:28<00:00, 11.16it/s, Loss=6.787e-03, PDE=1.777e-03, BC=1.298e-04,3.097e-06, IC=4.610e-03,2.674e-04, D*=6.00e-02,2.75e-03, lr=2.46e-05]


R^2 Error: {'Ni': 0.074, 'SiC': 0.01, 'C': 0.054, 'NiSi': 0.099, 'NiSi2': 0.053}
Inf Error: {'Ni': 0.071, 'SiC': 0.017, 'C': 0.111, 'NiSi': 0.11, 'NiSi2': 0.06}
Mean Error: (0.057934678911459356, 0.0736355660552526)
模型已保存至 ./Results/PINN_[64]x3/model_save/PINN_[64]x3_45000.pth
Epoch: 10/10----------------


100%|██████████| 5000/5000 [07:27<00:00, 11.17it/s, Loss=6.476e-03, PDE=1.707e-03, BC=1.250e-04,2.618e-06, IC=4.389e-03,2.524e-04, D*=6.00e-02,2.72e-03, lr=1.01e-08]


R^2 Error: {'Ni': 0.074, 'SiC': 0.01, 'C': 0.052, 'NiSi': 0.098, 'NiSi2': 0.053}
Inf Error: {'Ni': 0.07, 'SiC': 0.017, 'C': 0.108, 'NiSi': 0.11, 'NiSi2': 0.06}
Mean Error: (0.05736183888906825, 0.07295880528788676)
模型已保存至 ./Results/PINN_[64]x3/model_save/PINN_[64]x3_50000.pth
Training time: 4471.14s-------------------------


In [4]:
# from torch import Tensor


# hiddens = [6, 5, 4, 3]  ##[3,4] already finished
# neurons = [128,96,64]

# activation = torch.nn.functional.silu

# start_lr = 1e-3
# end_lr = 1e-8

# weight = [1, 1, 1, 1, 1, 1]
# for hidden in hiddens:
#     for neuron in neurons:
#         for mode in ['Fourier_PINN']:  ##['PINN', 'IA_PINN', 'Fourier_PINN', 'AF_PINN']
#             layer = [2] + [neuron]*hidden + [6]
            
#             star_time = timeit.default_timer()
#             num_epochs = 5000
#             sub_spochs = 10

#             loss_log_total,loss_log_pde, loss_log_bc, loss_log_ic = [],[],[],[]
#             torch.cuda.empty_cache()
#             print(f'{mode}_[{neuron}]x{hidden}:')
            
#             if mode == 'PINN':
#                 model = PINN(layer,activation,non_negative=True).to(device)
#                 optimizer = optim.Adam(model.parameters(), lr=start_lr)  
#                 scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * sub_spochs, eta_min=end_lr)
#             elif mode == 'IA_PINN':
#                 model = Attention_PINN(layer,activation,non_negative=True).to(device)
#                 optimizer = optim.Adam(model.parameters(), lr=start_lr)  
#                 scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * sub_spochs, eta_min=end_lr)
#             elif mode == 'Fourier_PINN':
#                 model = Fourier_PINN(layer,non_negative=True, activation= activation, use_rff=True, rff_num_features=64, rff_sigma=5.0).to(device)
#                 optimizer = optim.Adam(model.parameters(), lr=start_lr)  
#                 scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * sub_spochs, eta_min=end_lr)
#             elif mode == 'AF_PINN':
#                 model = AF_PINN(layer, non_negative=True,activation = activation, use_rff=True, rff_num_features=64, rff_sigma=5.0).to(device)
#                 optimizer = optim.Adam(model.parameters(), lr=start_lr)  
#                 scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * sub_spochs, eta_min=end_lr)
#             else:
#                 print("Man! What can I say! Wrong Mode!!!")
#                 break            
#             points_types = ['domain', 'left_bc', 'right_bc', 'initial']

#             coords_pde, coords_bc1, coords_bc2, coords_ic = [
#                 generator.grid_generator(2500 if pt in ['left_bc', 'right_bc'] else 10201, points_type=pt, nondim= True) 
#                 for pt in points_types
#             ]

#             for idx in range(sub_spochs):
#                 print(f'Epoch: {idx+1}/{sub_spochs}----------------')
#                 model.train()
                
#                 pbar = tqdm(range(num_epochs))
#                 for epoch in pbar:
#                     current_lr = optimizer.param_groups[0]['lr'] 
#                     def closure() -> Tensor:
#                         global loss_pde, D_star, loss_bc1, loss_bc2, loss_ic1, loss_ic2
#                         optimizer.zero_grad()

#                         # 计算损失时使用batch数据
#                         loss_eq1, loss_eq2, loss_eq3, loss_eq4, loss_eq5, loss_eq6, D_star = compute_pde(model, coords_pde)
#                         loss_pde = loss_eq1 + loss_eq2 + loss_eq3 + loss_eq4 + loss_eq5 + loss_eq6
#                         loss_bc1 = compute_bc(model, coords_bc1)
#                         loss_bc2 = compute_bc(model, coords_bc2)
#                         loss_ic1, loss_ic2 = compute_ic(model, coords_ic)

#                         #Give a weight to initial loss term
#                         loss = weight[0]*loss_pde +  weight[1]*loss_bc1 +  weight[2]*loss_bc2 +  weight[3]*loss_ic1 +  weight[4]*loss_ic2 

#                         loss.backward()
#                         torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0, norm_type= 2)
#                         return loss

#                     optimizer.step(closure)
                    
#                     # 如果使用scheduler（只在Adam阶段）
#                     if scheduler is not None:
#                         scheduler.step()
                    
#                     loss_total = weight[0]*loss_pde +  weight[1]*loss_bc1 +  weight[2]*loss_bc2 +  weight[3]*loss_ic1 +  weight[4]*loss_ic2 

#                     loss_log_total.append(loss_total.item())
#                     loss_log_pde.append(loss_pde.item())
#                     loss_log_bc.append(loss_bc1.item() + loss_bc2.item())
#                     loss_log_ic.append(loss_ic1.item() + loss_ic2.item())
                    
#                     if epoch % 10 == 0:
#                         pbar.set_postfix({
#                             'Loss': '{0:.3e}'.format(loss_total),
#                             'PDE': '{0:.3e}'.format(weight[0]*loss_pde.item()),
#                             'BC': '{0:.3e},{1:.3e}'.format(weight[1]*loss_bc1.item(),weight[2]*loss_bc2.item()),
#                             'IC': '{0:.3e},{1:.3e}'.format(weight[3]*loss_ic1.item(),weight[4]*loss_ic2.item()),
#                             'D*': '{0:.2e},{1:.2e}'.format(torch.max(D_star),torch.min(D_star)),
#                             'lr': '{0:.2e}'.format(current_lr),
#                         })        
#                     # # 将四个张量放在一个列表中并随机打乱
#                     shuffled_tensors = [tensor[torch.randperm(tensor.size(0))] for tensor in [coords_pde, coords_bc1, coords_bc2, coords_ic]]

#                     # 解包到四个变量中
#                     coords_pde, coords_bc1, coords_bc2, coords_ic = shuffled_tensors
                
#                     if torch.isnan(loss_total):
#                         print('Loss is Nan, please check your model parameters')
#                         break
                
#                 true_path = './FDM/range[0,1]/'
#                 file_names = ['u1_output.csv','u2_output.csv','u3_output.csv','u4_output.csv','u5_output.csv','u6_output.csv']
#                 true_data = [pd.read_csv(true_path + file_name,header=None) for file_name in file_names]

#                 model.eval()
#                 val_data = generator.grid_generator(N=10201, points_type= 'domain', nondim= True)
#                 result = model(val_data).data.cpu().numpy()
#                 relative_error(result,true_data)

#                 ##模型储存
#                 save_path = f'./Results/{mode}_[{neuron}]x{hidden}/model_save/'
#                 os.makedirs(save_path, exist_ok=True)  # 确保保存路径存在
#                 # 指定模型文件名
#                 model_file = os.path.join(save_path, f'{mode}_[{neuron}]x{hidden}_{(idx+1)*num_epochs}.pth')
#                 # 保存模型
#                 torch.save(model.state_dict(), model_file)
#                 print(f"模型已保存至 {model_file}")
            
#             # 绘制结果
#             plot_loss_curve(f'{mode}_[{neuron}]x{hidden}',loss_log_total,loss_log_pde, loss_log_bc, loss_log_ic)
            
#             plot_relative_concen(f'{mode}_[{neuron}]x{hidden}',result,data_save=True)
            
#             plot_3d_concen(f'{mode}_[{neuron}]x{hidden}', result, true_data)

#             Training_time = timeit.default_timer() - star_time
#             print(f'Training time: {Training_time:.2f}s-------------------------')
